In [1]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
np.random.seed(30)
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from collections import Counter
import keras
from keras.layers import Conv2D,Dense,Flatten
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Input
from keras.models import Model

/home/rithwik/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def network():
    
    input_layer = Input(shape=(4,7,1),name="Monthly_Matrix")
    
    

    op1 = Conv2D(40,kernel_size=(4,1), strides=(1,1),name="Filter_4X1")(input_layer)
    
    op1 = Flatten()(op1)
    
    
    
    op2 = Conv2D(40,kernel_size=(1,7), strides=(1,1),name="Filter_1X7")(input_layer)
    
    op2 = Flatten()(op2)
    
    
    op3 = Conv2D(40,kernel_size=(1,2),strides=(1,1),name="Filter_1X2")(input_layer)
    
    op3 = Flatten()(op3)
    
    op4 = Conv2D(40,kernel_size=(2,1),strides=(1,1),name="Filter_2X1")(input_layer)
    
    
    op4  = Flatten()(op4)
    
    
    
    merge_layer = keras.layers.concatenate([op1, op2, op3,op4],name="Concatenation_1")
    
    
    outputs = []
    
    for i in range(28):
        layer = Dense(100)(merge_layer)
        output_day = Dense(24,activation='softmax')(merge_layer)
        
        outputs.append(output_day)
        
    return Model(inputs=input_layer,outputs=outputs)

In [3]:
model = network()

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Monthly_Matrix (InputLayer)     (None, 4, 7, 1)      0                                            
__________________________________________________________________________________________________
Filter_4X1 (Conv2D)             (None, 1, 7, 40)     200         Monthly_Matrix[0][0]             
__________________________________________________________________________________________________
Filter_1X7 (Conv2D)             (None, 4, 1, 40)     320         Monthly_Matrix[0][0]             
__________________________________________________________________________________________________
Filter_1X2 (Conv2D)             (None, 4, 6, 40)     120         Monthly_Matrix[0][0]             
__________________________________________________________________________________________________
Filter_2X1

In [52]:
def load_data(string,n_months,path='../Datasets for Mid Sem Presentation/'):
    
    X = np.load(path+string+"_x.npy")
    y = np.load(path+string+"_y.npy")
    
    print (X.shape,y.shape)
    
    X = X.reshape((len(X),4,7))/(1000*1.0)
    y = y.reshape((len(y),4,168))/(1000*1.0)
    y = y.reshape((-1,168*4))
    X = np.expand_dims(X,axis=3)
    
    
    
    
    return X,y
        
    
    
    
def relative_cost(truth,pred):
    return np.mean(abs(pred-truth)/(.001+truth))




def relative_metric(truth,pred):
    return K.mean(abs(pred-truth)/(.001+truth))





def abs(truth,pred):
    return np.mean(1000*np.abs(truth-pred))

In [37]:
test_x,test_y = load_data('test_data',1)
train_x,train_y = load_data("train_data_1",3)
v_x,v_y = load_data("val_data_1",3)

((22, 28), (22, 672))
((258, 28), (258, 672))
((81, 28), (81, 672))


In [38]:
days_reading_test = np.sum(test_y.reshape((-1,28,24)),axis=2)[:,:,None]

In [39]:
days_reading_test.shape

(22, 28, 1)

In [40]:
def return_network_output(y):
    outputs = np.divide(y.reshape((-1,28,24)),np.sum(y.reshape((-1,28,24)),axis=2)[:,:,None])
    ans = []
    for i in range(28):
        ans.append(outputs[:,i,:])
        
    return ans

In [41]:
train_y = return_network_output(train_y)
test_y = return_network_output(test_y)
v_y = return_network_output(v_y)

In [48]:
dir_name = "Own Architecture"
model = network()

In [54]:

n_epochs = 1000
sgd = keras.optimizers.SGD(lr=0.01)
rmsprop  = keras.optimizers.RMSprop(lr=.001)
adagrad = keras.optimizers.adagrad(lr=.001)
model.compile(optimizer=sgd, loss='mean_squared_error')
filepath="weights.best.h5"
checkpoint = ModelCheckpoint(dir_name+'/single-model-{epoch:03d}.h5', verbose=5, monitor='val_loss',save_best_only=True, mode='auto')  
model.fit(train_x, train_y, epochs=n_epochs, validation_data=[v_x,v_y], callbacks=[checkpoint], verbose=5,batch_size=3)

Train on 258 samples, validate on 81 samples
Epoch 1/1000

Epoch 00001: val_loss improved from inf to 0.35646, saving model to Own Architecture/single-model-001.h5
Epoch 2/1000

Epoch 00002: val_loss improved from 0.35646 to 0.28821, saving model to Own Architecture/single-model-002.h5
Epoch 3/1000

Epoch 00003: val_loss improved from 0.28821 to 0.23981, saving model to Own Architecture/single-model-003.h5
Epoch 4/1000

Epoch 00004: val_loss improved from 0.23981 to 0.21427, saving model to Own Architecture/single-model-004.h5
Epoch 5/1000

Epoch 00005: val_loss improved from 0.21427 to 0.17923, saving model to Own Architecture/single-model-005.h5
Epoch 6/1000

Epoch 00006: val_loss improved from 0.17923 to 0.16189, saving model to Own Architecture/single-model-006.h5
Epoch 7/1000

Epoch 00007: val_loss improved from 0.16189 to 0.15204, saving model to Own Architecture/single-model-007.h5
Epoch 8/1000

Epoch 00008: val_loss improved from 0.15204 to 0.13001, saving model to Own Architec

In [42]:
print(abs(np.array(test_y)*days_reading_test,0))

ValueError: operands could not be broadcast together with shapes (28,22,24) (22,28,1) 

In [47]:
model.predict(test_x)

[array([[6.61913492e-03, 6.75123738e-05, 3.46028100e-05, 1.53921230e-03,
         8.19639742e-01, 4.64837393e-03, 2.18716432e-05, 1.14429175e-04,
         4.17051933e-06, 6.77610956e-07, 1.61193743e-01, 1.49309679e-04,
         2.48069664e-05, 3.10967956e-03, 2.78663239e-03, 2.99409386e-08,
         6.26952712e-08, 2.33354783e-07, 3.57141289e-05, 8.41975663e-08,
         1.19637559e-06, 3.56197870e-06, 6.50215506e-11, 5.19203286e-06],
        [7.98068568e-02, 8.94132908e-03, 1.15230251e-02, 5.87449372e-02,
         2.64319718e-01, 2.44924128e-02, 6.15780102e-03, 4.71592546e-02,
         4.08047391e-03, 8.67955945e-03, 2.85711318e-01, 1.12805665e-02,
         2.46723052e-02, 5.41354679e-02, 6.36768490e-02, 9.58617020e-04,
         1.25181838e-03, 1.91866711e-03, 2.39672661e-02, 8.76782287e-04,
         5.71112894e-03, 1.93193113e-03, 2.54417217e-04, 9.74749867e-03],
        [3.27865448e-04, 7.10966648e-04, 7.11609973e-05, 8.89225339e-04,
         7.16750979e-01, 1.03800101e-02, 4.963651